<a href="https://colab.research.google.com/github/eyupdalan/BLM6114-hw3/blob/main/BLM6114_hw3_large.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade fsspec==2025.3.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requir

In [2]:
!pip install -q transformers datasets peft trl accelerate bitsandbytes pandas openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 110.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Veri Setini Hazırlama

In [3]:
from typing import Literal
from datasets import Dataset
import pandas as pd

gpt4o = "gpt4o cevabı"
deepseek = "deepseek cevabı"

def generate_dataframes(type: Literal[gpt4o, deepseek]):
  print(f"generate_dataframes:Generating {type} dataframe...")
  excel_data_frame = pd.read_excel("ogrenci_sorular_2025.xlsx", engine="openpyxl")
  result = excel_data_frame[["Soru", type]].copy()
  result.columns=["soru", "cevap"]

  return result

def generate_dataset(type):
  print(f"generate_dataset:Generating {type} dataset...")
  df = generate_dataframes(type)
  df["text"] = "### Soru:\n" + df["soru"].astype(str) + "\n\n### Cevap:\n" + df["cevap"].astype(str)
  return Dataset.from_pandas(df[["text"]])



### Model Hazırlama

In [4]:
from typing import Literal
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from trl import SFTTrainer

medium = "ytu-ce-cosmos/turkish-gpt2-medium"
large = "ytu-ce-cosmos/turkish-gpt2-large"

def generate_model_and_tokenizer(model_id: Literal[medium, large]):
  print(f"generate_model_and_tokenizer:Generating {model_id}...")
  tokenizer = AutoTokenizer.from_pretrained(model_id)
  model = AutoModelForCausalLM.from_pretrained(model_id)

  print(f"{model_id} generated...")
  return model, tokenizer

def fine_tune_and_save_model(model_id: Literal[medium, large], data_type: Literal["gpt4o", "deepseek"], dataset):
  print(f"fine_tune_and_save_model:Fine tuning {model_id}...")
  model, tokenizer = generate_model_and_tokenizer(model_id)

  lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules="all-linear",
    modules_to_save=["lm_head", "embed_token"],
    task_type="CAUSAL_LM",
    #r=8,
    #lora_alpha=32,
    #target_modules=["c_attn"],
    #lora_dropout=0.05,
    #bias="none",
    #task_type=TaskType.CAUSAL_LM,
  )

  training_args = TrainingArguments(
    output_dir=f"./{model_id}-{data_type}-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    logging_dir=f"./{model_id}-{data_type}-logs",
    logging_steps=100,
    save_strategy="epoch",
    fp16=True,
    push_to_hub=False,
  )

  trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    #tokenizer=tokenizer,
    args=training_args,
    peft_config=lora_config,
    #dataset_text_field="text",
    #max_seq_length=512
  )

  trainer.train()
  print(f"{model_id}-{data_type} trained...")
  trainer.model.save_pretrained(f"./{model_id}-{data_type}-adapter")
  print(f"{model_id}-{data_type} model saved as ./{model_id}-{data_type}-adapter ...")
  trainer.tokenizer.save_pretrained(f"./{model_id}-{data_type}-adapter")
  print(f"{model_id}-{data_type} tokenizer saved as ./{model_id}-{data_type}-adapter ...")



### Kullanma

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

def get_model_to_use(model_id: Literal[medium, large],data_type: Literal["gpt4o", "deepseek"]):
  print(f"get_model_to_use:Getting {model_id}...")
  adapter = f"./{model_id}-{data_type}-adapter"
  base_model = AutoModelForCausalLM.from_pretrained(model_id)
  tokenizer = AutoTokenizer.from_pretrained(adapter)
  model = PeftModel.from_pretrained(base_model, adapter)
  model.eval()
  return model, tokenizer

def generate_pipeline(model_id: Literal[medium, large], data_type: Literal["gpt4o", "deepseek"]):
  print(f"generate_pipeline:Generating {model_id} pipeline...")
  model, tokenizer = get_model_to_use(model_id, data_type)
  pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
  return pipe

def run_prompt(question: str, pipe):
  print(f"run_prompt:Running {question}...")
  prompt = f"""
  ### Soru:
  {question}

  ### Cevap:
  """
  result = pipe(prompt, max_new_tokens=100, do_sample=True, temperature=0.7, top_p=0.95)
  return result[0]["generated_text"]



### Kontroller

##### Model-Adapter Kaydetme

In [6]:
gpt_dataset = generate_dataset(gpt4o)
deepseek_dataset = generate_dataset(deepseek)

#fine_tune_and_save_model(medium,"gpt4o",gpt_dataset)
#fine_tune_and_save_model(medium,"deepseek", deepseek_dataset)
fine_tune_and_save_model(large,"gpt4o",gpt_dataset)
fine_tune_and_save_model(large,"deepseek", deepseek_dataset)

generate_dataset:Generating gpt4o cevabı dataset...
generate_dataframes:Generating gpt4o cevabı dataframe...
generate_dataset:Generating deepseek cevabı dataset...
generate_dataframes:Generating deepseek cevabı dataframe...
fine_tune_and_save_model:Fine tuning ytu-ce-cosmos/turkish-gpt2-large...
generate_model_and_tokenizer:Generating ytu-ce-cosmos/turkish-gpt2-large...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/537 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/927k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/585k [00:00<?, ?B/s]

...

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss
100,2.599000
200,2.427900
300,2.350500
400,2.329100
500,2.293700
600,2.274700
700,2.272600
800,2.209900
900,2.179100
1000,2.062000


ytu-ce-cosmos/turkish-gpt2-large-deepseek trained...


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


ytu-ce-cosmos/turkish-gpt2-large-deepseek model saved as ./ytu-ce-cosmos/turkish-gpt2-large-deepseek-adapter ...
ytu-ce-cosmos/turkish-gpt2-large-deepseek tokenizer saved as ./ytu-ce-cosmos/turkish-gpt2-large-deepseek-adapter ...


In [10]:
pipe_gpt_large = generate_pipeline(large, "gpt4o")
pipe_deepseek_large = generate_pipeline(large, "deepseek")

result_gpt_large = run_prompt("zebra mı daha hızlıdır at mı?", pipe_gpt_large)
print("result_gpt_large",result_gpt_large)

result_deepseek_large = run_prompt("zebra mı daha hızlıdır at mı?", pipe_deepseek_large)
print("result_deepseek_large",result_deepseek_large)

generate_pipeline:Generating ytu-ce-cosmos/turkish-gpt2-large pipeline...
get_model_to_use:Getting ytu-ce-cosmos/turkish-gpt2-large...


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

generate_pipeline:Generating ytu-ce-cosmos/turkish-gpt2-large pipeline...
get_model_to_use:Getting ytu-ce-cosmos/turkish-gpt2-large...


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoFo

run_prompt:Running zebra mı daha hızlıdır at mı?...
result_gpt_large 
  ### Soru:
  zebra mı daha hızlıdır at mı?

  ### Cevap:
  
**Zebra** daha hızlıdır ve daha hızlı koşabilir.  
Ama **at** daha hızlıdır.  
**Zebra**, hızlı olduğu için daha fazla mesafe kat edebilir.  
**At** ise, daha yavaş olduğu için daha fazla hız kazanamaz.  

Yani zebra daha hızlı, at daha yavaşdır.  
Yani **zebra** daha hızlıdır, çünkü **at** daha hızlı koşar.  
**At** ise, **zebra**
run_prompt:Running zebra mı daha hızlıdır at mı?...
result_deepseek_large 
  ### Soru:
  zebra mı daha hızlıdır at mı?

  ### Cevap:
  
Zebra: Daha hızlıdır, çünkü atlar daha hızlı koşabilir.  
At: Daha yavaş, çünkü atlar daha hızlı koşamaz.  
Hız: Atlar, zebralar gibi koşamaz.  
Sonuç:  
Atlar hızlı koşabilir, çünkü atların koşma yeteneği atlara özgüdür.  
Atlar daha hızlı koşabilir, çünkü atlar daha hızlı koşabilir.


In [11]:
import pandas as pd

# Load questions from the file
with open("yeni_sorular_50_adet.txt", "r", encoding="utf-8") as f:
    questions = f.readlines()
    questions = [q.strip() for q in questions if q.strip()]  # Remove empty lines

# Function to run prompt and get responses
def get_responses(question):
  result_gpt_large = run_prompt(question, pipe_gpt_large)
  result_deepseek_large = run_prompt(question, pipe_deepseek_large)
  return result_gpt_large, result_deepseek_large

# Generate and print responses for all questions
all_responses = []
for question in questions:
  result_gpt_large, result_deepseek_large = get_responses(question)
  print(f"Question: {question}")
  print(f"GPT Large Response: {result_gpt_large}")
  print(f"Deepseek Large Response: {result_deepseek_large}")
  print("-" * 50)

  all_responses.append({
      "question": question,
      "gpt_response_large": result_gpt_large,
      "deepseek_response_large": result_deepseek_large,
  })

# Optionally save responses to a file
df = pd.DataFrame(all_responses)
df.to_excel("responses.xlsx", index=False)
print("Responses saved to responses.xlsx")

run_prompt:Running Şu nesnelerin ortak özelliği nedir? Kalem, silgi, cetvel...
run_prompt:Running Şu nesnelerin ortak özelliği nedir? Kalem, silgi, cetvel...
Question: Şu nesnelerin ortak özelliği nedir? Kalem, silgi, cetvel
GPT Large Response: 
  ### Soru:
  Şu nesnelerin ortak özelliği nedir? Kalem, silgi, cetvel

  ### Cevap:
   - Kalem, silgi, cetvel, cetvel

   - **Ortak Özellik**:

- Kalem: Yazma ve çizim aracıdır.  
- Silgi: Kalem gibi sert, ince ve yumuşaktır.  
- **Ortak Özellik**:

- **Kalem**: **Kalem**, **silgi** ve **defter** gibi yazılı araçlardır.  
- **Defter**: **Defter**, **not** veya **not** gibi yazılı
Deepseek Large Response: 
  ### Soru:
  Şu nesnelerin ortak özelliği nedir? Kalem, silgi, cetvel

  ### Cevap:
  
1. **Kalem**:
   - Fiziksel olarak kalem, yazı yazmayı sağlar.
   - Elle yazılan yazılardan daha dayanıklıdır.
   - Yazı yazma, çizme ve boyama gibi birçok işlevi yerine getirir.
  
2. **Silgi**:
   - Yazı yazmak için kullanılır.
   - Elle yazılan yazılard

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Question: Şu listedeki eylemler hangi ortamda gerçekleşir? Yazmak, okumak, çizmek
GPT Large Response: 
  ### Soru:
  Şu listedeki eylemler hangi ortamda gerçekleşir? Yazmak, okumak, çizmek

  ### Cevap:
   - **Yazmak**, **okumak** ve ** çizmek** birer **olgu**dur. Bu eylemler **ortam**da gerçekleşir.
Deepseek Large Response: 
  ### Soru:
  Şu listedeki eylemler hangi ortamda gerçekleşir? Yazmak, okumak, çizmek

  ### Cevap:
   - **Yazmak**: Kalemle veya dijital araçlarla yazı yazmak.  
   - **Okumak**: Yazılı materyallerle etkileşim kurmak.  
   - **Yazmak**: Duygu, düşünce veya bilgileri aktarmak.  
   - **Çizmek**: Nesnelerin, karakterlerin veya çizimlerin canlandırılması.  
   - **Yazmak**: Görselleştirmek (yazıda veya çizimde).  
   - **Çizmek**: Bir resmi veya çizimi dijital araçlarla dijital
--------------------------------------------------
run_prompt:Running Güneş, yıldız, ay bu gök cisimlerinin ortak özelliği nedir?...
run_prompt:Running Güneş, yıldız, ay bu gök cisimlerinin o